In [1]:
import os
import torch
import numpy as np

print(os.sys.path)
os.sys.path.append("/home/matthias/DIKU Project Protein Folding/protein-folding-gans/src/openprotein/pnerf")

# CODE FILES HERE
from model_params import get_model_data_dcgan
from models.dcgan import Dcgan, Generator, Discriminator
from solver import Solver
from directories import Directories
from dataloader import DataLoader
from plots import plot_losses, plot_z_samples
from sampling import dcgan_sampling
from openprotein.preprocessing import process_raw_data

# SETTINGS HERE
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # to see the CUDA stack
%matplotlib inline
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
# supress cluttering warnings in solutions
import warnings
warnings.filterwarnings('ignore')

['/home/matthias/DIKU Project Protein Folding/protein-folding-gans/src', '/home/matthias/.local/share/virtualenvs/openprotein-xj-YAKcS/lib/python37.zip', '/home/matthias/.local/share/virtualenvs/openprotein-xj-YAKcS/lib/python3.7', '/home/matthias/.local/share/virtualenvs/openprotein-xj-YAKcS/lib/python3.7/lib-dynload', '/home/matthias/anaconda3/envs/openprotein/lib/python3.7', '', '/home/matthias/.local/share/virtualenvs/openprotein-xj-YAKcS/lib/python3.7/site-packages', '/home/matthias/.local/share/virtualenvs/openprotein-xj-YAKcS/lib/python3.7/site-packages/IPython/extensions', '/home/matthias/.ipython']


In [2]:
# setting device on GPU if available, else CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cpu



In [3]:
dataset_arg = "proteins"
data = get_model_data_dcgan(dataset_arg)
residue_fragments = 16

In [4]:
training_file = "openprotein/data/preprocessed/testing.txt.hdf5"
validation_file = "openprotein/data/preprocessed/testing.txt.hdf5"
#process_raw_data(False, force_pre_processing_overwrite=False)

In [ ]:
directories = Directories("dcgan", dataset_arg.lower(), data["z_dim"], make_dirs=False)
data_loader = DataLoader(directories, data["batch_size"], dataset_arg.lower(),
                         training_file=training_file, validation_file=validation_file,
                         residue_fragments=residue_fragments, atom="calpha")
gan = Dcgan(data_loader.input_dim, data["z_dim"])
generator = Generator(data["z_dim"], data_loader.input_dim, data_loader.img_dims, res=residue_fragments)
discriminator = Discriminator(data_loader.max_sequence_length()//data_loader.residue_fragments,\
                              1, res=residue_fragments)
solver = Solver(gan, generator, discriminator, data["epochs"], data_loader, data["optimizer_G"],
                data["optimizer_D"], data["optim_config_G"], data["optim_config_D"], save_model_state=False)
solver.main()

(16, 16) 256 2 2 81 2 81
+++++ START RUN +++++ | no save mode
====> Epoch: 1 g_loss / d_loss avg: 0.5787 / 0.5557
47.03 seconds for epoch 1
====> Epoch: 2 g_loss / d_loss avg: 0.8753 / 0.3086
51.14 seconds for epoch 2
====> Epoch: 3 g_loss / d_loss avg: 0.9303 / 0.2872
48.90 seconds for epoch 3
====> Epoch: 4 g_loss / d_loss avg: 1.4482 / 0.1515
44.99 seconds for epoch 4
====> Epoch: 5 g_loss / d_loss avg: 1.7896 / 0.1078
46.31 seconds for epoch 5
====> Epoch: 6 g_loss / d_loss avg: 2.1735 / 0.0746
45.42 seconds for epoch 6
====> Epoch: 7 g_loss / d_loss avg: 2.1327 / 0.0765
45.90 seconds for epoch 7
====> Epoch: 8 g_loss / d_loss avg: 2.4744 / 0.0573
45.74 seconds for epoch 8
====> Epoch: 9 g_loss / d_loss avg: 2.3940 / 0.0731
46.54 seconds for epoch 9
====> Epoch: 10 g_loss / d_loss avg: 2.1803 / 0.0961
46.16 seconds for epoch 10
====> Epoch: 11 g_loss / d_loss avg: 2.6851 / 0.0485
45.57 seconds for epoch 11
====> Epoch: 12 g_loss / d_loss avg: 2.6553 / 0.0585
45.10 seconds for epoch

In [ ]:
# Insert name of model here if want to load a model
#solver = torch.load(solver.save_model_dir + "/model_GAN_mnist_z=100.pt")
#solver.model.eval()
#solver.data_loader.directories.make_dirs = False

In [ ]:
# Plotting g and d losses for all epochs
plot_losses(solver, solver.train_loss_history["g_loss"], solver.train_loss_history["d_loss"])

In [ ]:
samples = gan_sampling(generator, solver.model.z_dim, 100)
plot_z_samples(samples, solver.data_loader.directories.make_dirs,
               solver.data_loader.directories.result_dir, solver.data_loader.dataset)